<style>
    .jp-OutputArea-output {
        display: flex;
        justify-content: center;
        width: 50vw; /* Default width for laptops and monitors */
    }

    .jp-Cell {
        display: flex;
        justify-content: center;
    }

    .jp-Notebook {
        align-items: center;
    }

    @media (max-width: 768px) {
        .jp-OutputArea-output {
            width: 95vw; /* Width for small screens */
        }
    }

    @media (min-width: 769px) and (max-width: 1200px) {
        .jp-OutputArea-output {
            width: 50vw; /* Width for laptops */
        }
    }
</style>


## **CACA TRACKER**

In [1]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Cargar los datos del JSON
with open('caca_log.json', 'r') as f:
    data = json.load(f)

In [2]:
# Convertir los datos a un DataFrame de pandas
df = pd.DataFrame(data)
df['fecha'] = pd.to_datetime(df['fecha'])

In [3]:
# Calcular la frecuencia de cacas por persona y ordenar de mayor a menor
frecuencia = df['nombre'].value_counts().reset_index()
frecuencia.columns = ['nombre', 'count']
frecuencia = frecuencia.sort_values('count', ascending=True)

# Crear un mapa de colores para cada persona
color_map = px.colors.qualitative.Plotly[:len(frecuencia)]

# Crear la gráfica de barras
fig_frecuencia = px.bar(
    frecuencia, 
    x='count', 
    y='nombre', 
    orientation='h',
    labels={'nombre': 'Nombre', 'count': 'Número de cacas'},
    title='Frecuencia de cacas por persona',
    color='nombre',
    color_discrete_sequence=color_map
)

# Actualizar el diseño para mejorar la apariencia y espaciar los nombres
fig_frecuencia.update_layout(
    showlegend=False,
    yaxis={
        'categoryorder': 'total ascending',
        'tickmode': 'array',
        'tickvals': frecuencia['nombre'],
        'ticktext': [f'  {nombre}  ' for nombre in frecuencia['nombre']]
    }
)

# Mostrar la figura
fig_frecuencia.show()

In [4]:
# 2. Gráfica circular: "Distribución de tipos de caca"
fig_tipos = px.pie(df['tipo'].value_counts().reset_index(), 
                   values='count', names='tipo',
                   title='Distribución de tipos de caca')
fig_tipos.show()

In [5]:
# 3. Gráfica de líneas: "Tendencia de cacas por día"
df_diario = df.groupby(df['fecha'].dt.date).size().reset_index(name='count')
fig_tendencia = px.line(df_diario, x='fecha', y='count',
                        labels={'fecha': 'Fecha', 'count': 'Número de cacas'},
                        title='Tendencia de cacas por día')
fig_tendencia.show()

In [6]:
# Obtener la lista de personas únicas
personas = df['nombre'].unique()

# Crear un DataFrame con todas las fechas y personas
fecha_min = df['fecha'].min().date()
fecha_max = df['fecha'].max().date()
todas_fechas = pd.date_range(fecha_min, fecha_max, freq='D')
df_completo = pd.DataFrame([(fecha, persona) for fecha in todas_fechas for persona in personas],
                           columns=['fecha', 'nombre'])

# Contar las cacas por día y persona
df_conteo = df.groupby([df['fecha'].dt.date, 'nombre']).size().reset_index(name='count')
df_conteo['fecha'] = pd.to_datetime(df_conteo['fecha'])  # Convertir a datetime

# Combinar el DataFrame completo con el conteo
df_final = pd.merge(df_completo, df_conteo, on=['fecha', 'nombre'], how='left').fillna(0)

# Crear la figura
fig = go.Figure()

# Añadir una línea para cada persona
for persona in personas:
    df_persona = df_final[df_final['nombre'] == persona]
    
    fig.add_trace(
        go.Scatter(x=df_persona['fecha'], y=df_persona['count'], name=persona, mode='lines+markers')
    )

# Actualizar el diseño de la figura
fig.update_layout(
    title="Tendencia de cacas por día y por persona",
    xaxis_title="Fecha",
    yaxis_title="Número de cacas",
    legend_title="Personas"
)

# Actualizar los ejes y
fig.update_yaxes(range=[0, df_final['count'].max() + 1])

# Mostrar la figura
fig.show()

In [7]:
# 7. Gráfica de barras apiladas: "Distribución de cacas por hora del día"
def get_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Mañana'
    elif 12 <= hour < 18:
        return 'Tarde'
    elif 18 <= hour < 24:
        return 'Noche'
    else:
        return 'Madrugada'

df['time_of_day'] = df['fecha'].dt.hour.apply(get_time_of_day)
fig_distribucion = px.histogram(df, x='time_of_day', color='nombre',
                                category_orders={'time_of_day': ['Madrugada', 'Mañana', 'Tarde', 'Noche']},
                                title='Distribución de cacas por hora del día')
fig_distribucion.show()

In [8]:
# 9. Gráfica de dispersión: "Mapa de calor de cacas"
df['dia_semana'] = df['fecha'].dt.dayofweek
df['hora'] = df['fecha'].dt.hour
heatmap_data = df.groupby(['dia_semana', 'hora']).size().reset_index(name='count')
fig_heatmap = px.density_heatmap(heatmap_data, x='dia_semana', y='hora', z='count',
                                 labels={'dia_semana': 'Día de la semana', 'hora': 'Hora del día', 'count': 'Número de cacas'},
                                 title='Mapa de calor de cacas')
fig_heatmap.show()

### Final Stats

In [9]:
# 4. Métrica divertida: "El Rey de la Porcelana"
rey_porcelana = df['nombre'].value_counts().index[0]
num_cacas_rey = df['nombre'].value_counts().max()
print(f"{rey_porcelana} es el Rey de la Porcelana con {num_cacas_rey} visitas al trono.")

Angel es el Rey de la Porcelana con 38 visitas al trono.


In [10]:
# 5. Métrica divertida: "El Madrugador"
df['hora'] = df['fecha'].dt.hour + df['fecha'].dt.minute / 60
madrugador = df.loc[df['hora'].idxmin()]
print(f"{madrugador['nombre']} es El Madrugador con una caca registrada a las {madrugador['fecha'].strftime('%H:%M')}.")

Alex es El Madrugador con una caca registrada a las 00:20.


In [11]:
# 6. Métrica divertida: "El Noctámbulo"
noctambulo = df.loc[df['hora'].idxmax()]
print(f"{noctambulo['nombre']} es El Noctámbulo con una caca registrada a las {noctambulo['fecha'].strftime('%H:%M')}.")

Angel es El Noctámbulo con una caca registrada a las 23:30.


In [12]:
# 8. Métrica divertida: "El Más Consistente"
df['minutos_del_dia'] = df['fecha'].dt.hour * 60 + df['fecha'].dt.minute
consistencia = df.groupby('nombre')['minutos_del_dia'].std().sort_values()
mas_consistente = consistencia.index[0]
print(f"{mas_consistente} es El Más Consistente con una desviación estándar de {consistencia[0]:.2f} minutos en sus horarios de caca.")

Luis es El Más Consistente con una desviación estándar de 177.26 minutos en sus horarios de caca.


In [13]:
# 10. Métrica divertida: "El Campeón de la Regularidad"
df_sorted = df.sort_values('fecha')
df_sorted['diff'] = df_sorted.groupby('nombre')['fecha'].diff().dt.days
df_sorted['streak'] = (df_sorted['diff'] != 1).groupby(df_sorted['nombre']).cumsum()
max_streak = df_sorted.groupby(['nombre', 'streak']).size().groupby(level=0).max()
campeon_regularidad = max_streak.idxmax()
dias_consecutivos = max_streak.max()
print(f"{campeon_regularidad} es El Campeón de la Regularidad con {dias_consecutivos} días consecutivos de cacas.")

Angel es El Campeón de la Regularidad con 3 días consecutivos de cacas.
